# **ProboNAS example**

##### Drive and libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Insert inside ***your_path*** the path to the code folder in your drive:

In [ ]:
your_path = ""

In [ ]:
import os

for f in os.listdir('/content/drive/MyDrive/'+your_path+'Notebooks/'):
  os.system(f"cp -r /content/drive/MyDrive/"+your_path+"Notebooks/{f} .")

In [ ]:
!pip install -U 'git+https://github.com/facebookresearch/fvcore'
!pip install pyvww

In [ ]:
import torch
import torchvision
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from torchvision import transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset

import shutil

from pyvww.pytorch import VisualWakeWordsClassification

import numpy as np
import pandas as pd

from invbneckzpadd import InvBNeck

from convnextzpadd import ConvNext

from Downsampling import Downsampling

from createNetStem import Net
from measures import *
from measures.logsynflow import compute_synflow_per_weight
from measures.naswot import compute_naswot_score
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import parameter_count


from randomNetStem import randomGen
import random
import time

from PIL import Image

from genetic_algorithmStem import ProboNAS

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

##### Prepare the data folder

Insert inside ***train_path*** the path to the tar file that contains the train data in your drive (example: '/content/drive/MyDrive/Project4a/dataset/train2014.tar'):

In [ ]:
!mkdir train2014
!tar -xf train_path -C train2014

Insert inside ***val_path*** the path to the tar file that contains the validation data in your drive (example: '/content/drive/MyDrive/Project4a/dataset/val2014.tar'):

In [ ]:
!mkdir val2014
!tar -xf val_path -C val2014

In [ ]:
#define a function & pass dst. directory and src. directories
def merge_directories(new_directory_name, *directories_to_merge):
    if not os.path.exists(new_directory_name):
        os.makedirs(new_directory_name) #create a dst. directory if not exist

    for directory in directories_to_merge:
        for item in os.listdir(directory):  #iterate sub-directory from source folders
            #join path of folder and sub-folder
            s = os.path.join(directory, item)
            d = os.path.join(new_directory_name, item)
            if os.path.isdir(s):
                if item in os.listdir(new_directory_name):
                    files = os.listdir(s)
                    for file in files:  #iterate file from sub-folder
                        j = os.path.join(s, file)
                        k = os.path.join(d, file)
                        shutil.copy2(j,k)  #paste file in already existed sub-directory
                else:
                    shutil.copytree(s, d)  #create a sub-directory in dst directory then paste file
            else:
                shutil.copy2(s, d)  #paste file in already existed sub-directory

!mkdir all2014
dst_directory = '/content/all2014'
src_directory1 = "/content/train2014/content/path-to-COCO-dataset/train2014"
src_directory2 = "/content/val2014/content/path-to-COCO-dataset/val2014"

#call function and pass path of the dst directory & all src directories.
merge_directories(dst_directory, src_directory1, src_directory2)

In [ ]:
folder = '/content/all2014'

for file_name in os.listdir(folder):
	oldName = os.path.join(folder, file_name)
	n = os.path.splitext(file_name)[0]
	n = n.split('_')[2]
	b = n + '.jpg'
	newName = os.path.join(folder, b)
  # Rename the file
	os.rename(oldName, newName)

##### Create dataset and dataloader

Insert inside ***data_path*** the path to the dataset folder in your drive:

In [ ]:
data_path = ""

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.Resize(128)])

batch_size = 32

trainset = VisualWakeWordsClassification(root="/content/all2014",
                                         annFile="/content/drive/MyDrive/"+data_path+"annotations2014/instances_train.json",
                                         transform=transform
                                         )
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = VisualWakeWordsClassification(root="/content/all2014",
                                        annFile="/content/drive/MyDrive/"+data_path+"annotations2014/instances_val.json",
                                        transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('person', 'not-person')

##### ProboNAS

In [ ]:
"""Singole probabilità che geni/blocchi cambino"""
params={"p_block":1,
        "p_gene": 0.3,
        "std_channels": 5}

import warnings
warnings.filterwarnings('ignore')

genetic_code=ProboNAS(N=25, n=5, trainloader=trainloader, max_time=15, max_params=2500000, max_flops=200000000*batch_size, params=params)

In [ ]:
genetic_code

In [ ]:
net=Net(3,2,genetic_code, is_training=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
net.to(device)

import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.PolynomialLR(optimizer, total_iters = 8, power = 1)

losses =[]
net.train()
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        losses.append(loss.item())
        if i %  100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    scheduler.step()

print('Finished Training')

dataiter = iter(testloader)
images, labels = next(dataiter)


plt.plot(losses)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

y_pred = []
y_true = []

net.eval()
# iterate over test data
for inputs, labels in tqdm(testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    output = net(inputs) # Feed Network

    output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
    y_pred.extend(output) # Save Prediction

    labels = labels.data.cpu().numpy()
    y_true.extend(labels) # Save Truth

# constant for classes
classes = ('person', 'not-person')

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')

In [ ]:
import sklearn

sklearn.metrics.accuracy_score(y_true, y_pred)